__Install conda and load modules__

After installing [miniconda](https://conda.io/miniconda.html), install the following packages:

matplotlib numpy scipy astropy pandas yt xarray healpy

by typing the following into the command prompt

> $conda install matplotlib numpy scipy astropy pandas yt xarray [healpy]

anything else..?

* __I've been using python 2.7.x but let's try to stick to the python 3.x standard.__


* __tigradpy is a rewrite of pyathena. Currently, it is a submodule only to the radiation (and ionrad-dev) branch...much work has to be done__

In [ ]:
from __future__ import print_function
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import yt

yt.funcs.mylog.setLevel(50) # suppress log messages except for critical ones
# add tigradpy to python path..
sys.path.insert(0, '../../')
sys.path.insert(0, '../../pyathena') # add Chang-Goo's pyathena to python path
import tigradpy as tp
import pyathena as pa
print(tp.__all__)

__Read simulation input parameters using read_athinput__

In [ ]:
#datadir = '/tigress/jk11/tigrad_pp/rad_8pc_pp_dmax1024/'
datadir = '/scratch/gpfs/jk11/radps_postproc/rad_8pc_dmax1024_nproc8/'
#datadir = '/home/jk11/athena-tigress/bin/'
fathinput = os.path.join(datadir, 'out.txt')
par = tp.read_athinput(fathinput) # par is a namedtuple object

print(par._fields)

print(type(par.problem)) # each item of par is a dictionary
# problem block
print('\n','<problem>')
for k, v in par.problem.iteritems():
    print('*',k,':',v)
print('\n','<radps>')
# radps block
for k, v in par.radps.iteritems():
    print('*',k,':',v)
print('\n','<configure>')
# seems like some entries in configure block are not read properly
for k, v in par.configure.iteritems():
    print('*',k,':',v)

__Read vtk file using yt and add derived fields__

In [ ]:
## this is very inconvenient..need to write a conveience unit object
import yt.units as yu
from yt import physical_constants as phyc
units_override = dict(length_unit=(1., 'pc'),
                      time_unit=((yu.pc/(yu.km/yu.s)).in_units('Myr'), 'Myr'),
                      mass_unit=((1.4272*phyc.mass_hydrogen_cgs*
                                 (yu.pc/yu.cm)**3).in_units('Msun'), 'Msun'))

num = 1
#datadir = '/home/jk11/athena-tigress/bin/'
#datadir = '/scratch/gpfs/jk11/radps_postproc/rad_8pc_dmax1024_nproc8/'
datadir = '/tigress/jk11/radps_postproc/rad_8pc_dmax1024_nproc8/'

filename = os.path.join(datadir, 'rad_8pc.{0:04d}.vtk'.format(num))

##Somehow yt spits out a strange warning message when dataset is loaded..
# ds = yt.load(filename, units_override=units_override)

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ds = yt.load(filename, units_override=units_override)

tp.add_fields(ds, IXN=3)

__all vtk output variables are in code unit.__

In [ ]:
ds.field_list

__Let's see how derived fields can be added to yt dataset.__

In [ ]:
# %load ../add_fields.py

__Example x-z and x-y slices and projections of $n_{\rm H}$, $x_n$, radiation densities__

G0prime1: FUV Mean intensity normalized by solar neighborhood value
\begin{equation}
    {\rm G0prime1} = J_{\rm FUV}/(2.1 \times 10^{-4}\,{\rm erg}\,{\rm cm}^{-2}\,{\rm s}^{-1}\,{\rm sr}^{-1})
\end{equation}

Similarly, define EUV mean intensity as
\begin{equation}
    {\rm G0prime0} = J_{\rm EUV}/(2.1 \times 10^{-4}\,{\rm erg}\,{\rm cm}^{-2}\,{\rm s}^{-1}\,{\rm sr}^{-1})
\end{equation}

In [ ]:
# fields = ['nH', 'xn', 'G0prime0', 'G0prime1']

# sy = tp.yt_multipanel(ds, 'slice', 'y', fields=fields)
# py = tp.yt_multipanel(ds, 'projection', 'y', fields=fields)
# sz = tp.yt_multipanel(ds, 'slice', 'z', fields=fields)
# pz = tp.yt_multipanel(ds, 'projection', 'z', fields=fields)

__Plot multiple snapshots and save__

In [2]:
from __future__ import print_function
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import yt
import yt.units as yu
from yt import physical_constants as phyc

yt.funcs.mylog.setLevel(50) # suppress log messages except for critical ones
sys.path.insert(0, '../../')
import tigradpy as tp

units_override = dict(length_unit=(1., 'pc'),
                      time_unit=((yu.pc/(yu.km/yu.s)).in_units('Myr'), 'Myr'),
                      mass_unit=((1.4272*phyc.mass_hydrogen_cgs*
                                 (yu.pc/yu.cm)**3).in_units('Msun'), 'Msun'))

fields = ['nH', 'xn', 'G0prime0', 'G0prime1']
# kind = (['slice','y'],['slice','z'],
#         ['projection','y'],['projection','z'])
kind = [['slice','y']]

num = range(1, 2, 1)
datadir = '/tigress/jk11/radps_postproc/rad_8pc_dmax1024/'
savdir = os.path.join('/tigress/jk11/snapshots-rad_8pc_2/')
if not os.path.exists(savdir):
    os.makedirs(savdir)
    
for num_ in num:
    print(num_, end=' ')
    filename = os.path.join(datadir, 'rad_8pc.{0:04d}.vtk'.format(num_))
    ds_ = yt.load(filename, units_override=units_override)
    tp.add_fields(ds_, IXN=3)
    for kind_, axis_ in kind:
        fname = '{0:s}_{1:s}_{2:04d}.png'.format(kind_, axis_, num_)
        p_ = tp.yt_multipanel(ds_, kind_, axis_, fields=fields)
        # add time stamp and save
        fig = p_.plots[fields[0]].figure # reference to the mpl figure
        ax = fig.axes[0]
        ax.set_title('time {0:5.1f}'.format(ds_.current_time))
        fig.savefig(os.path.join(savdir, fname))
        plt.close(fig)

1 

__midplane slices from raw arrays__

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.flatten()

Nx1 = par.domain1['Nx1']
Nx2 = par.domain1['Nx2']
Nx3 = par.domain1['Nx3']
ix = Nx1
iy = Nx2
iz = Nx3/2

num = 5
# read joined vtk..
datadir = '/scratch/gpfs/jk11/tigrad_pp/rad_8pc_pp_dmax1024/'
filename = os.path.join(datadir, 'rad_8pc_pp.{0:04d}.vtk'.format(num))
ds = yt.load(filename, units_override=units_override)
tp.add_fields(ds, IXN=3)

da = ds.all_data()
den = da['nH'].reshape(Nx1,Nx2,Nx3)[:,:,iz].v
T = da['Temperature'].reshape(Nx1,Nx2,Nx3)[:,:,iz].v
#Tma = np.ma.masked_where(T > 1e4, T)

plt.sca(axes[0])
im0 = plt.imshow(np.log10(den).T, origin='lower', vmin=-3, vmax=3,
                 cmap='Spectral_r')
divider = make_axes_locatable(plt.gca())
cax0 = divider.append_axes('right', size='5%', pad=0.05)
cb0 = plt.colorbar(im0, cax=cax0)
cb0.set_label(r'$\log_{10}\,n_{\rm H}$')

plt.sca(axes[1])
im1 = plt.imshow(np.log10(T).T, origin='lower', vmin=1.0, vmax=6,
                 cmap='RdYlBu_r')
divider = make_axes_locatable(plt.gca())
cax1 = divider.append_axes('right', size='5%', pad=0.05)
cb1 = plt.colorbar(im1, cax=cax1)
cb1.set_label(r'$\log_{10}\,T$')

plt.sca(axes[2])
xn = da['xn'].reshape(Nx1,Nx2,Nx3)[:,:,iz].v
im2 = plt.imshow(np.log10(xn).T, origin='lower', vmin=-8.0, vmax=0.0,
                 cmap='viridis')
divider = make_axes_locatable(plt.gca())
cax2 = divider.append_axes('right', size='5%', pad=0.05)
cb2 = plt.colorbar(im2, cax=cax2)
cb2.set_label(r'$\log_{10}\,x_{\rm n}$')


num = 5
num_orig = int(par.problem['num_start'] + (num-1)*(par.problem['num_stride']))

datadir_orig = '/tigress//jk11/tigrad_pp/rad_8pc'
#datadir = '/home/jk11/athena-tigress/bin'
filename = os.path.join(datadir_orig, 'rad_8pc.{0:04d}.vtk'.format(num_orig))

##############################################################
## Read original data (before postprocessing) using pyathena
## (bottom right panel)
##############################################################

import pyathena as pa
import astropy.constants as ac
import astropy.units as au

ds_orig = pa.AthenaDataSet(filename)
den3d = ds_orig.read_all_data('density')
P = ds_orig.read_all_data('pressure')
unit = pa.set_units(muH=1.4272)
to_pok=(unit['pressure']/ac.k_B).cgs.value
pok = P*to_pok
T1_new = pok/den/1.4272
T1 = ds_orig.read_all_data('T1')
temp = pa.coolftn.get_temp(T1)
den_orig = den3d[112,:,:]
T_orig = temp[112,:,:]

plt.sca(axes[3])
im1 = plt.imshow(np.log10(T_orig), origin='lower', vmin=1.0, vmax=6,
                 cmap='RdYlBu_r')
divider = make_axes_locatable(plt.gca())
cax1 = divider.append_axes('right', size='5%', pad=0.05)
cb1 = plt.colorbar(im1, cax=cax1)
cb1.set_label(r'$\log_{10}\,T$')
plt.tight_layout()

##### Compare temperature before and after post-processing

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
plt.sca(axes[0])
plt.scatter(den_orig.flatten(), T_orig.flatten(), alpha=0.05, s=1, c='r')
plt.scatter(den.flatten(), T.flatten(), alpha=0.03, s=1)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('n_H')
plt.ylabel('T (after postproc) [K]')
plt.xlim(1e-5, 1e3)

plt.sca(axes[1])
plt.scatter(T_orig.flatten(), T.T.flatten(), alpha=0.2, s=1)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('T (before) [K]')
plt.ylabel('T (after postproc) [K]')
#plt.xlim(1e-5,1e2)

__Read zprof__

In [ ]:
#datadir = '/scratch/gpfs/jk11/radps_postproc/rad_8pc_dmax1024/'
#datadir = '/home/jk11/athena-tigress/bin'
datadir = '/tigress/jk11/radps_postproc/rad_8pc_dmax1024_nproc8/'
num = 5
filename = os.path.join(datadir, 'id0', 
                        'rad_8pc.{0:04d}.whole.zprof'.format(num))
zp = tp.read_zprof(filename)
print(zp.columns)
plt.plot(zp.z, zp.Erad0, 'r-', label=r'$\langle\mathcal{E}_{\rm EUV}\rangle$')
plt.plot(zp.z, zp.Erad1, 'b-', label=r'$\langle\mathcal{E}_{\rm FUV}\rangle$')
plt.yscale('log')
plt.xlabel('z [pc]')
plt.legend(fontsize='x-large')

__Read all zprof__

In [ ]:
zpa = tp.read_zprof_all(os.path.join(datadir, 'id0'), 'rad_8pc',
                        phase='whole', force_override=True)
print(type(zpa))
zpa.coords, zpa.fields

In [ ]:
#print(zpa.loc['Erad0'].shape)
Erad0 = zpa.loc['Erad0']
Erad1 = zpa.loc['Erad1']
xn = zpa.loc['s4']/zpa.loc['d']
#print(np.array(zpa.time))
#print('{0:16.10e}'.format(float(zpa.time[4])))

In [ ]:
import matplotlib as mpl

plt.figure()
print(zpa.z.min(), zpa.z.max())
extent = (200, 400, zpa.z.min(), zpa.z.max())
aspect = 0.08
print(extent)
fig, axes = plt.subplots(3, 1, figsize=(8, 15), sharex=True, gridspec_kw=dict(hspace=0.0))
plt.sca(axes[0])
plt.imshow(Erad0, extent=extent, aspect=aspect)
plt.sca(axes[1])
plt.imshow(Erad1, extent=extent, aspect=aspect)
plt.sca(axes[2])
plt.imshow(xn, extent=extent, vmin=0, vmax=1, aspect=aspect)

for ax in axes:
    ax.set_ylabel('z [pc]')
    
axes[2].set_xlabel('time [Myr]')

* __3d vtk can be joined using shell script__


* __For full details, type athena-tigress/vtk/join.sh -h__


* __See also Python parallel wrapper script__

In [ ]:
import subprocess
subprocess.call(['../../../vtk/join.sh','-r','1:7','-i',datadir])